In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [47]:
import datasets

dataset = datasets.load_dataset("wikisql")

  0%|          | 0/3 [00:00<?, ?it/s]

In [63]:
trainDataSet=dataset["train"]
testDataSet=dataset["test"]

In [83]:
train_questions = trainDataSet["question"]
train_categories= [ sql["agg"] for sql in trainDataSet["sql"]]

train_cleaned_questions = [q[:-1] if q.endswith("?") else q for q in train_questions]
train_cleaned_questions = [q.replace("\xa0", " ") for q in train_cleaned_questions]
train_cleaned_questions = [q.strip() for q in train_cleaned_questions]

test_questions = testDataSet["question"]
test_categories= [ sql["agg"] for sql in testDataSet["sql"]]

test_cleaned_questions = [q[:-1] if q.endswith("?") else q for q in test_questions]
test_cleaned_questions = [q.replace("\xa0", " ") for q in test_cleaned_questions]
test_cleaned_questions = [q.strip() for q in test_cleaned_questions]


s="What's the total number of picks for the player Matt Murton"
try:
    index = train_cleaned_questions.index(s)
    print(f"The string 'banana' is found at index {index}.")
except ValueError:
    print("The string 'banana' does not exist in the list.")
    

The string 'banana' is found at index 53504.


In [84]:
train_cleaned_questions[53504]

"What's the total number of picks for the player Matt Murton"

In [65]:
from numpy import asarray
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

In [66]:
def get_keras_model():
    """Define the model."""
    model = Sequential()
    model.add(Dense(128, input_shape=[512 ,], activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64 ,activation='relu' ,kernel_regularizer=tf.keras.regularizers.L1(0.01),
                    activity_regularizer=tf.keras.regularizers.L2(0.01)))
    model.add(Dense(6, activation='softmax'))

    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    return model

In [67]:
#x_train, x_test, y_train ,y_test =train_test_split(cleaned_questions, categories, shuffle=False)



In [68]:
import tensorflow as tf
import tensorflow_hub as hub

embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [69]:
def get_embeddings(x):
    embeddings = embed(x)
    return asarray(embeddings)


In [11]:
#train_encodings = get_embeddings(x_train.to_list())
#test_encodings = get_embeddings(x_test.tolist())

In [73]:
#train_encodings = get_embeddings(x_train)
#test_encodings = get_embeddings(x_test)
train_encodings = get_embeddings(train_cleaned_questions)
test_encodings = get_embeddings(test_cleaned_questions)


In [74]:
y_train = asarray(train_categories, dtype="float32")
y_test = asarray(test_categories, dtype="float32")


In [75]:

model = get_keras_model()
print(train_encodings.shape)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               65664     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 6)                 390       
                                                                 
Total params: 74,310
Trainable params: 74,310
Non-trainable params: 0
_________________________________________________________________
(56355, 512)


In [76]:
model.fit(train_encodings, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
1409/1409 [==============================] - 12s 7ms/step - loss: 1.2009 - accuracy: 0.7433 - val_loss: 0.8200 - val_accuracy: 0.7452
Epoch 2/50
1409/1409 [==============================] - 9s 7ms/step - loss: 0.7388 - accuracy: 0.7823 - val_loss: 0.7584 - val_accuracy: 0.7668
Epoch 3/50
1409/1409 [==============================] - 10s 7ms/step - loss: 0.6913 - accuracy: 0.7961 - val_loss: 0.7262 - val_accuracy: 0.7736
Epoch 4/50
1409/1409 [==============================] - 11s 8ms/step - loss: 0.6632 - accuracy: 0.8048 - val_loss: 0.7024 - val_accuracy: 0.7845
Epoch 5/50
1409/1409 [==============================] - 10s 7ms/step - loss: 0.6457 - accuracy: 0.8116 - val_loss: 0.6846 - val_accuracy: 0.7988
Epoch 6/50
1409/1409 [==============================] - 11s 8ms/step - loss: 0.6313 - accuracy: 0.8180 - val_loss: 0.6738 - val_accuracy: 0.8003
Epoch 7/50
1409/1409 [==============================] - 10s 7ms/step - loss: 0.6170 - accuracy: 0.8262 - val_loss: 0.6576 - val_acc

In [77]:
model.save("Question_Classifier.h5")

In [78]:
# testing
score, acc = model.evaluate(test_encodings, y_test)

497/497 [==============================] - 2s 3ms/step - loss: 0.5519 - accuracy: 0.8593
